In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time
import random

# Simulation Project V2 (FA23)

The purpose of this notebook is to further modularize simulation model v1 and incorporate our own twists to examine the behavior of gender-based bias in the corporate workplace.

In [2]:
# Global Variables / References:

# Starting index for workers- used to measure seniority
worker_id = 1

# number-char correspondences for pretty printing
work_levels = {
    1 : "J",
    2 : "M",
    3 : "S",
    4 : "E"
}

worker_genders = {
    0 : "M",
    1 : "F"
}

# Appropriate staff sizes for each level:
level_sizes = {
    1 : 400,
    2 : 100,
    3 : 25,
    4 : 5
}

# The impact that seniority has on staying time - older people retire later?
# Set negative for opposite effect.
level_ext_stay_times = {
    1 : 0,
    2 : 10,
    3 : 20,
    4 : 30
}

# The impact that gender has on staying time -- women pushed out quicker?
gender_stay_times = {
    0 : 30,
    1 : 28
}

# A database to store workers, organized by level:
# Implemented as simple lists, ordered from most to least senior by employee index.
worker_db = {
    1 : [],
    2 : [],
    3 : [],
    4 : []
}

In [3]:
# Worker Class:
class worker(object):
    
    def __init__(self, level, gender, idx, start_time):
        self.level = level
        self.gender = gender
        self.idx = idx
        self.start_time = start_time
        
    def __lt__(self, other):
        return self.idx < other.get_index()
        
    def __str__(self):
        return "[level: %s, gender: %s, id: %i, end_time: %f]" \
            % (self.level, self.gender, self.idx, self.end_time)
        
    def get_index(self):
        return self.idx
    
    def get_level(self):
        return self.level
    
    def set_end_time(self):
#       Setting end time for male employees
        if self.gender == 0:
            self.end_time = self.start_time \
                + np.random.exponential( \
                gender_stay_times[self.gender] + level_ext_stay_times[self.level])
            
#       Setting end time for female employees - min of 2 Expo RVs
        else:
            self.end_time = self.start_time \
                + min(np.random.exponential( \
                gender_stay_times[self.gender] + level_ext_stay_times[self.level]), \
                gender_stay_times[self.gender] + level_ext_stay_times[self.level])       
                
    def get_end_time(self):
        return self.end_time
    
    def get_gender(self):
        return self.gender
    
    def promote(self):
        self.level += 1
        self.set_end_time()

In [12]:
def remove_expired_workers(worker_db):
    for level in worker_db.keys():
        
        l = len(worker_db[level])
        idx = 0
        
#       keys are being changed during the loop- for loop won't work!
        while idx < l:
#           if a worker has 'expired', kick 'em out!
            if worker_db[level][idx].get_end_time() < time.time():
                worker_db[level] = worker_db[level][:idx] \
                                    + worker_db[level][idx+1:]
#               reflect that the size of the level is one less after removal
                l -= 1
            idx += 1
            
#       Sort to ensure most senior employees are promoted first
        worker_db[level].sort()

def hire_worker(worker_db, level, idx, all_male=False, all_female=False):
    assert not all_female or not all_male
    # Randomized gender for new hire -- Change to 3 evenutally for NB case?
    if all_female:
        gender = 1
    elif all_male:
        gender = 0
    else:
        gender = random.randint(0,1)

    # Create worker:
    hire = worker(level, gender, idx, time.time())
    hire.set_end_time()
    

    # Add worker to worker database:     
    worker_db[hire.get_level()].append(hire)
    
    
def promote_workers(worker_db):
    # Promote to fill ranks:           
    level = max(worker_db.keys())
#   Promote employees to fill levels 2-4. Work from top to ensure each level is full.
    while level > 1:
        while len(worker_db[level]) < level_sizes[level]:
#           promote most senior employee from one level down:
            worker_db[level].append(worker_db[level-1][0])
    
#           adjust level, end time for promoted employee
            worker_db[level][len(worker_db[level]) - 1].promote()
            worker_db[level][len(worker_db[level]) - 1].set_end_time()
        
#           remove employee from level below
            worker_db[level-1] = worker_db[level-1][1:]
        level -= 1

def populate_initial_workforce(start_id, all_male = False, all_female = False):
    
    for key in worker_db.keys():
        worker_db[key] = []
    
    level = 4
    idx = 1
    
    while level >= 1:
        while len(worker_db[level]) < level_sizes[level]:
            hire_worker(level, idx, all_male, all_female)
        
            idx += 1 
            
        level -= 1
    return idx 


In [49]:
class Simulation(object):
    """
    A class to streamline the simulation process. 
    """
    
    def __init__(self, worker_db, level_sizes, leaving_process, hiring_process, promotion_process, 
                 round_length=30, num_rounds=6):
        
        self.worker_db = worker_db
        self.level_sizes = level_sizes
        self.leaving_process = leaving_process
        self.hiring_process = hiring_process
        self.promotion_process = promotion_process
        self.round_length = round_length
        self.num_rounds = num_rounds
        self.worker_idx = 1
        
    def populate_initial_workforce(self, all_male=False, all_female=False):
        for key in worker_db.keys():
            worker_db[key] = []

        level = max(work_levels.keys())

        while level >= 1:
            for i in range(self.level_sizes[level]):
                self.hiring_process(self.worker_db, level, self.worker_idx, all_male, all_female)

                self.worker_idx += 1 

            level -= 1
                
    def hire_entry_workers(self, worker_db):
        level = max(worker_db.keys())
        
        while level >= 1:
            while len(worker_db[level]) < level_sizes[level]:
                self.hiring_process(self.worker_db, level, self.worker_idx)
                self.worker_idx += 1
            level -= 1
            
            
    
    def update_workforce(self):
        # Delete expired workers:
        self.leaving_process(self.worker_db)
        
        # Promote to fill ranks:  
        self.promotion_process(self.worker_db)
        
        # Add new hires to fill out staff:
        self.hire_entry_workers(self.worker_db)
        
    def overall_gender_distribution(self,db):
        count_male = 0
        count_female = 0
        total = 0

        for level in db.keys():
            for wrkr in db[level]:
                if wrkr.get_gender() == 0:
                    count_male += 1
                else:
                    count_female += 1
                total += 1

        print("population male: %i, population female: %i, total: %i \n" % (count_male, count_female, total))

    def level_gender_distribution(self, worker_level):
        count_male = 0
        count_female = 0
        total = 0

        for wrkr in worker_level:
            if wrkr.get_gender() == 0:
                count_male += 1
            else:
                count_female += 1
            total += 1

        pct_male = count_male
        pct_female = count_female

        return print("population male: %i, population female: %i, total: %i \n" % (count_male, count_female, total))

    def print_current_demo_info(self):
        self.overall_gender_distribution(self.worker_db)
    
    def print_final_demo_info(self):
        for level in self.worker_db.keys():
            self.level_gender_distribution(self.worker_db[level])
    
    def run_simulation(self):
        start_time = time.time()
        
        self.populate_initial_workforce()
        
        print("Pre-Simulation Demographic Info:\n")
        
        print("Overall Gender Distribution:\n")
        self.print_current_demo_info()
        
        print("Level-Split Gender Distribution:\n")
        self.print_final_demo_info()
        
        print("Startng Simulation: %i rounds of %i seconds each." % (self.num_rounds, self.round_length))
        
        for i in range(self.num_rounds):
            print("starting new round")
            # for duration of simulation length:
            while time.time() <= start_time + self.round_length:
                self.update_workforce()
                time.sleep(.5)
                if (time.time() - start_time) % (self.round_length/5) < .5: 
                    pct_complete = min(100,round(((time.time() - start_time) / self.round_length*100)))
                    print("round %i %i%s complete" % (i+1, pct_complete, "%"))

            print("round complete.")
            self.print_current_demo_info()
            
            start_time = time.time()
        
        print("Simulation Complete. Final Stats:\n")
        self.print_final_demo_info()
        

In [50]:
basic_sim = Simulation(worker_db, level_sizes, remove_expired_workers,hire_worker, promote_workers, round_length=10)

basic_sim.run_simulation()

Pre-Simulation Demographic Info:

Overall Gender Distribution:

population male: 253, population female: 277, total: 530 

Level-Split Gender Distribution:

population male: 190, population female: 210, total: 400 

population male: 50, population female: 50, total: 100 

population male: 10, population female: 15, total: 25 

population male: 3, population female: 2, total: 5 

Startng Simulation: 6 rounds of 10 seconds each.
starting new round
round 1 20% complete
round 1 40% complete
round 1 61% complete
round 1 81% complete
round 1 100% complete
round complete.
population male: 253, population female: 277, total: 530 

starting new round
round 2 20% complete
round 2 40% complete
round 2 61% complete
round 2 81% complete
round 2 100% complete
round complete.
population male: 254, population female: 276, total: 530 

starting new round
round 3 20% complete
round 3 40% complete
round 3 60% complete
round 3 81% complete
round 3 100% complete
round complete.
population male: 277, popula